In [13]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import os
from pptx import Presentation

from pptx.util import Inches, Pt

from pptx.enum.shapes import MSO_SHAPE

from pptx.chart.data import ChartData

采用网厅3月数据，曹莉提供，没有任何人工修改——这一点很重要

In [14]:
def load_datafile(path):
    files=os.listdir(path)
    filesname=[]
    for s in files:
        filesname.append(s.split('.')[0])
    
    path_default=os.getcwd()
    os.chdir(path)
    data={}
    for i in range(len(files)):
        data[filesname[i]]=pd.read_excel(files[i],sheetname=None)
    
    os.chdir(path_default)
    return filesname,data

In [15]:
def Index_to_list(df_index):
    '''index:df.index'''
    index_list=[]
    for p in df_index:
        index_list.append(p)
    return index_list

数据到ppt，as chart

In [16]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches
def df_change_to_chart(df,shape):
    '''
    用df替换原来的数据，其他不变
    df:data,type=DataFrame
    shape:have a chart,we replace data from df
    '''
    chart_data=ChartData()
    chart_data.categories=df.columns
    for p in df.index:
        chart_data.add_series(p,df.T[p])
    shape.chart.replace_data(chart_data)
def df_addto_shapes_chart(df,shapes,x=Inches(2), y=Inches(2), cx=Inches(6), cy=Inches(4.5),\
                          chart_type=XL_CHART_TYPE.COLUMN_CLUSTERED):
    '''
    把df加到shapes中，作为chart，位置为x, y, cx, cy
    chart类型为XL_CHART_TYPE.COLUMN_CLUSTERED
    df:data,type=DataFrame
    shapes:we add df to shapes
    '''
    chart_data=ChartData()
    chart_data.categories=df.columns
    for p in df.index:
        chart_data.add_series(p,df.T[p])
    shapes.add_chart(
    chart_type, x, y, cx, cy, chart_data
)


def add_slide_df_chart(df,prs,slide_layout_number=0,x=Inches(2), y=Inches(2), cx=Inches(6), cy=Inches(4.5),\
                       chart_type=XL_CHART_TYPE.COLUMN_CLUSTERED):
    '''
    增加一页ppt，把df加到其shapes中，作为chart，位置为x, y, cx, cy
    chart类型为XL_CHART_TYPE.COLUMN_CLUSTERED
    df:data,type=DataFrame
    prs:Presentation
    slide_layout_number=0（不使用任何模板时，6为空白页）
    !!!!注意slide_layout_number上限与所取ppt有关
    '''
    slide_layout = prs.slide_layouts[slide_layout_number]
    slide = prs.slides.add_slide(slide_layout)
    shapes=slide.shapes
    df_addto_shapes_chart(df,shapes,x, y, cx, cy,\
                          chart_type)
    

数据处理

In [19]:
def data_to_df_vs(data,files,sheet,column,newcolumns):
    '''
    讲不同文件中相同sheet中相同列做比较,-df.index相同
    data
    files=[filesname[2],filesname[3]],type(filesname)=list
    sheet=u'Province Table'
    column=u'Page Views'
    newcolumns=chart_categories_name=[u'2月', u'3月']:新df.columns,len(newcolumns)不确定
    '''
    datavs={}
    for i in range(len(newcolumns)):
        datavs[newcolumns[i]]=data[files[i]][sheet][column]
    df_index=data[files[i]][sheet].index
    return DataFrame(datavs,index=df_index)

In [1]:
import os
import shutil

def get_newtestfile(path=u'2016-02月报end.pptx',path_backup=u'2016-02月报end - 副本 (2).pptx'):
    if os.path.isfile(path):
        os.remove(path)
    shutil.copyfile(path_backup,path)